In [1]:
import os

import openmm as mm
from openmm.app import *
from openmm import unit
from openmm import app
from pdbfixer import PDBFixer
import mdtraj as md
import nglview as nv
import requests
import numpy as np
from math import sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt

# Barnase-Barstar Simulation

## System Analysis

In [2]:
# Function to download PDB files

def fetch_pdb(pdb_id, download_path="./"):

        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)

In [3]:
# Just write the PDB id in order to download the pdb file

fetch_pdb("1brs")

In [4]:
# Load pdb with MDTraj

brs = md.load('1brs.pdb')

In [5]:
# Remove water molecules

brs = brs.remove_solvent()

In [6]:
# Barnase's chains

atoms_in_chain_A = brs.topology.select("chainid == 0")
atoms_in_chain_B = brs.topology.select("chainid == 1")
atoms_in_chain_C = brs.topology.select("chainid == 2")

In [7]:
# Barstar's chains

atoms_in_chain_D = brs.topology.select("chainid == 3")
atoms_in_chain_E = brs.topology.select("chainid == 4")
atoms_in_chain_F = brs.topology.select("chainid == 5")

In [8]:
# Barnase's chain's atoms

barnase_A = brs.atom_slice(atoms_in_chain_A)
barnase_B = brs.atom_slice(atoms_in_chain_B)
barnase_C = brs.atom_slice(atoms_in_chain_C)

In [9]:
# Barstar's chain's atoms

barstar_D = brs.atom_slice(atoms_in_chain_D)
barstar_E = brs.atom_slice(atoms_in_chain_E)
barstar_F = brs.atom_slice(atoms_in_chain_F)

<div class="alert alert-info">
<strong>NOTE:</strong> We now that chain B and F will be optimal to work.
</div>


In [10]:
view = nv.show_mdtraj(brs)
view

NGLWidget()

<div class="alert alert-info">
<strong>NOTE:</strong> Superpose F with E.
</div>

## System Preparation

In [50]:
atoms_to_fit_E = []
atoms_to_fit_F = []
for atom_E in barstar_E.topology.atoms_by_name('CA'):
    for atom_F in barstar_F.topology.atoms_by_name('CA'):
        if str(atom_E)==str(atom_F):
            print(f'{str(atom_E)} is {atom_E.index} in E and {atom_F.index} in F')
            atoms_to_fit_E.append(atom_E.index)
            atoms_to_fit_F.append(atom_F.index)

LYS2-CA is 1 in E and 10 in F
ALA3-CA is 10 in E and 19 in F
VAL4-CA is 15 in E and 24 in F
ILE5-CA is 22 in E and 31 in F
ASN6-CA is 30 in E and 39 in F
GLY7-CA is 38 in E and 47 in F
GLU8-CA is 42 in E and 51 in F
GLN9-CA is 51 in E and 60 in F
ILE10-CA is 60 in E and 69 in F
ARG11-CA is 68 in E and 77 in F
SER12-CA is 75 in E and 88 in F
ILE13-CA is 81 in E and 94 in F
SER14-CA is 89 in E and 102 in F
ASP15-CA is 95 in E and 108 in F
LEU16-CA is 103 in E and 116 in F
HIS17-CA is 111 in E and 124 in F
GLN18-CA is 121 in E and 134 in F
THR19-CA is 126 in E and 143 in F
LEU20-CA is 133 in E and 150 in F
LYS21-CA is 141 in E and 158 in F
LYS22-CA is 150 in E and 167 in F
GLU23-CA is 159 in E and 173 in F
LEU24-CA is 168 in E and 182 in F
ALA25-CA is 176 in E and 190 in F
LEU26-CA is 181 in E and 195 in F
PRO27-CA is 189 in E and 203 in F
GLU28-CA is 196 in E and 210 in F
TYR29-CA is 205 in E and 215 in F
TYR30-CA is 217 in E and 227 in F
GLY31-CA is 229 in E and 239 in F
GLU32-CA is 233

In [52]:
barstar_F_over_E = md.Trajectory.superpose(barstar_F, barstar_E, frame=0, atom_indices=atoms_to_fit_F, ref_atom_indices=atoms_to_fit_E)

In [53]:
pp = nv.show_mdtraj(barstar_F_over_E)
pp

NGLWidget()

In [55]:
new_barnase_barstar = barnase_B.stack(barstar_F_over_E)

In [56]:
uu = nv.show_mdtraj(new_barnase_barstar)
uu

NGLWidget()

In [57]:
new_barnase_barstar.save_pdb('new_barnase_barstar.pdb')

In [59]:
fixer = PDBFixer(filename='new_barnase_barstar.pdb')

In [60]:
fixer.topology

<Topology; 2 chains, 199 residues, 1577 atoms, 1611 bonds>

In [61]:
fixer.findMissingResidues()
missing_residues = fixer.missingResidues
print(f"{len(missing_residues)} missing residues")

fixer.findNonstandardResidues()
nonstandard_residues = fixer.nonstandardResidues
print(f"{len(nonstandard_residues)} non standard residues")

fixer.findMissingAtoms()
missing_atoms = fixer.missingAtoms
missing_terminals = fixer.missingTerminals
print(f"{len(missing_atoms)} missing atoms")
print(f"{len(missing_terminals)} missing terminals")

if len(nonstandard_residues)>0:
    fixer.replaceNonstandardResidues()

if len(missing_atoms)>0:
    fixer.addMissingAtoms()

0 missing residues
0 non standard residues
6 missing atoms
0 missing terminals


In [91]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [92]:
modeller = app.Modeller(fixer.topology, fixer.positions)
pH = 7.2
residues_protonated = modeller.addHydrogens(forcefield=forcefield, pH=pH)

In [104]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff, constraints=HBonds)

In [105]:
for ii in range(system.getNumForces()):
    print(system.getForce(ii).getForceGroup())

0
0
0
0
0


In [106]:
forcegroups = {}
for ii in range(system.getNumForces()):
    force = system.getForce(ii)
    force.setForceGroup(ii)
    forcegroups[force] = ii

In [107]:
for ii in range(system.getNumForces()):
    print(system.getForce(ii).getForceGroup())

0
1
2
3
4


In [108]:
step_size   = 0.002*unit.picoseconds
temperature = 0.0*unit.kelvin
friction    = 0.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

In [109]:
# Creación de la plataforma.

platform_name = 'CUDA'
platform    = mm.Platform.getPlatformByName(platform_name)

In [110]:
# Creación del objeto simulacion del sistema en vacio

simulation = Simulation(modeller.topology, system, integrator, platform)

In [111]:
# Condiciones iniciales
simulation.context.setPositions(modeller.positions)

In [112]:
# Minimizacion del sistema

state_pre_minimization = simulation.context.getState(getEnergy=True)
simulation.minimizeEnergy()
state_post_minimization = simulation.context.getState(getEnergy=True)

In [113]:
print('Energy before minimization:', state_pre_minimization.getPotentialEnergy())
print('Energy after minimization:', state_post_minimization.getPotentialEnergy())

Energy before minimization: -13231.263671875 kJ/mol
Energy after minimization: -25090.421875 kJ/mol


In [100]:
system = simulation.system

In [114]:
forcegroups

{<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7ff431dab7b0> >: 0,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7ff3dcbdf630> >: 1,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7ff3dcbdf5d0> >: 2,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7ff3dcbdf2a0> >: 3,
 <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7ff3dcbdf390> >: 4}

In [116]:
simulation.context.getState(getEnergy=True).getPotentialEnergy()

Quantity(value=-25090.421875, unit=kilojoule/mole)

In [122]:
energies = {}
for ff, ii in forcegroups.items():
    energies[ff.getName()] = simulation.context.getState(getEnergy=True, groups={ii}).getPotentialEnergy()

In [123]:
energies

{'HarmonicBondForce': Quantity(value=410.28607177734375, unit=kilojoule/mole),
 'HarmonicAngleForce': Quantity(value=1699.28955078125, unit=kilojoule/mole),
 'NonbondedForce': Quantity(value=-36737.25, unit=kilojoule/mole),
 'PeriodicTorsionForce': Quantity(value=9537.2509765625, unit=kilojoule/mole),
 'CMMotionRemover': Quantity(value=0.0, unit=kilojoule/mole)}

In [125]:
energies['HarmonicBondForce']+energies['HarmonicAngleForce']+energies['NonbondedForce']+energies['PeriodicTorsionForce']

Quantity(value=-25090.423400878906, unit=kilojoule/mole)

# Of interest

https://github.com/openmm/openmm/issues/1463    
https://github.com/openmm/openmm/issues/387    
https://github.com/openmm/openmm/issues/2682    
https://github.com/openmm/openmm/issues/1830    
https://openmm.github.io/openmm-cookbook/dev/notebooks/cookbook/Analyzing%20Energy%20Contributions.html    
https://openmm.github.io/openmm-cookbook/dev/notebooks/cookbook/Querying%20Charges%20and%20Other%20Parameters.html    


https://github.com/openmm/openmm/issues/3178    
https://github.com/openmm/openmm/issues/3169    
https://github.com/openmm/openmm/issues/2880    
https://github.com/openmm/openmm/issues/2835    
https://github.com/openmm/openmm/issues/2682    

https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/60c758a8567dfe4abcec68b3/original/sbm-open-mm-a-builder-of-structure-based-models-for-open-mm.pdf